## **Few-shot prompting**

Few-shot prompting involves giving language models examples within the prompt. We create a prompt with example question-answer pairs and the question we want the model to answer.

We fed this prompt to the model, and we get the answer to our question in return. With this approach, the model learns how to answer the given question from the examples.

<img src='./images/few-shots-prompting.png' width=60% height=60%>

* Number of examples:
  * Zero → Zero-shot prompting
  * One → One-shot prompting
  * More than one → Few-shot prompting

```python
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "Today the weather is fantastic"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I don't like your attitude"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That shot selection was awful"}
    ],
    temperature=0,
)
print(response.choices[0].message.content)
```

Output: <br>
`negative`

## **Multi-step prompting**

__Writing a travel blog post__

<div style="display: flex;">
    <!-- Left Column -->
    <div style="width: 45%; padding: 10px;">
        <ul>
            <li>Break down an end gol into series of steps</li>
            <li>Model goes through each step to improve accuracy for the final output</li>
            <li>Multi-step prompts are used for:</li>
            <ul>
                <li>Sequential tasks; like generating coherent text from an outline</li>
                <li>Cognitive tasks; such as evaluating a solution's correctness, as theyinvolve problem-solving and decision-making processes</li>
            </ul>
        </ul>
    </div>
    <!-- Right Column -->
    <div style="width: 48%; padding: 10px;">
        <img src='./images/multi-step-prompt.png' width=25%>
    </div>
</div>

```python
prompt= """Compose a travel blog as follows:
Step 1: Introduce the destination.
Step 2: Share personal adventures during the trip.
Step 3: Summarize the journey.
"""

print(get_response(prompt))
```
Output: <br>
<img src='./images/travel-blog-output.png' width=75% height=75%>

__Analyzing solution correctness__

As a starting point, we may start with a single-step prompt asking the model to assess the code's correctness. However, a simple 'yes' doesn't explain the evaluation. We may need specific criteria or domain knowledge to refine the evaluation. In this case, while the syntax is correct, the divide function doesn't handle division by zero, which should be considered in the evaluation.

```python
calculator = """
def add(a, b):
return a + b
def subtract(a, b):
return a - b
def multiply(a, b):
return a * b
def divide(a, b):
return a / b
"""

prompt = f"""Determine the correctness of the code delimited by triple backticks as follows:
Step 1: Check the code correctness in each function.
Step 2: Verify if the divide function handles the case when dividing by 0.
Code:
```{calculator}```"""

print(get_response(prompt))
```
output: <br>
<img src='./images/calculator-output.png' width=75% height=75%>

__Multi-step vs. few-shot prompt__

<div style="display: flex;">
    <!-- Left Column -->
    <div style="width: 45%; padding: 10px;">
        <div style="text-align: center;">
            Multi-step prompt
        </div>
        <ul>
            <li>Explicitly tell model what to do. They act as a roadmap for the model by providing specific guidance.</li>
        </ul>
        <div style="text-align: center;">
            <img src='./images/multi-step-prompt-full.png' width=25%>
        </div>
    </div>
    <!-- Right Column -->
    <div style="width: 45%; padding: 10px;">
        <div style="text-align: center;">
            Few-shot prompt
        </div>
        <ul>
            <li>Questions and answets model loeans from. They demonstrate how the model should respond to certain inputs. The model observes these examples and learns to generalize from them.</li>
        </ul>
        <div style="text-align: center;">
            <img src='./images/few-shot-prompt.png' width=24%>
        </div>
    </div>
</div>

### **Chain-of-thought and self-consistency prompting**

<div style="display: flex;">
    <!-- Left Column -->
    <div style="width: 45%; padding: 10px;">
        <ul>
            <li>Requires LLMs to provide reasoning steps (thoughts) before giving answer</li>
            <li>Used for complex reasoning tasks</li>
            <li>Help reduce model errors</li>
        </ul>
    </div>
    <!-- Right Column -->
    <div style="width: 45%; padding: 10px;">
        <img src='./images/chain-of-thought-prompt.png' width=24%>
    </div>
</div>


<div style="display: flex;">
    <!-- Left Column -->
    <div style="width: 45%; padding: 10px;">
        <b>Standard prompting to solve a reasoning task</b>
        <pre><code>
prompt = """Q: You start with 15 books in your collection. At the bookstore, you purchase 8 new books. Then, you lend 3 to your friend and 2 to your cousin. Later, you visit another bookstore and buy 5 more books. How many books do you have now?
A: The answer is"""
    
print(get_response(prompt))
</code></pre>
    <img src='./images/25-books-reasoning.png'>
    </div>
    <!-- Right Column -->
    <div style="width: 45%; padding: 10px;">
        <b>Chain-of-thought prompting to solve a reasoning task</b>
        <pre><code>
prompt = """Q: You start with 15 books in your collection. At the bookstore, you purchase 8 new books. Then, you lend 3 to your friend and 2 to your cousin. Later, you visit another bookstore and buy 5 more books. How many books do you have now?
A: Let's think step by step"""

print(get_response(prompt))
</code></pre>
    <img src='./images/25-books.png'>
    </div>
</div>

### **Chain-of-thought prompting with few-shots**

```python
example = """"
Q: The odd numbers in this group add up to an even number: 9, 10, 13, 4, 2.
A: Adding all the odd numbers (9, 13) gives 22. The answer is True.
"""

question = """Q: The odd numbers in this group add up to an even number: 15, 13, 82, 7.
A:
"""

prompt = example + question
print(get_response(prompt))
```
Output: <br>
`Adding all the odd numbers (15,13,7) gives 35. The answer is False.`

While multi-step prompts incorporate steps _inside the prompt_, Chain-of-thought prompts _ask_ model to generate intermediate steps.

### **Limitation**

* one unsuccessful thought → unsuccesfull outcome. <br>

<img src='./images/unsuccessful.png' width=15%>

### **Self-consistency prompting**

* Generates multiple chain-of-thoughts by prompting the model several times
* Majority vote to obtain final output

<img src='./images/multiple-chain-of-thought.png' width=35%>

Can be done by defining multiple prompts or a prompt generating multiple responses.

```python
self_consistency_instruction = "Imagine three completely independent experts who reason differently are answering this question. The final answer is obtained by majority vote. The question is: "

problem_to_solve= "If there are 10 cars in the parking lot and 3 more cars arrive. Half the original number of cars leave. Then, half of the current number of cars arrive. How many cars are there in the parking?"

prompt = self_consistency_instruction + problem_to_solve
print(get_response(prompt))
```

Output: <br>
<img src='./images/self-consistency-output.png' width=60%>

## **Iterative prompt engineering and refinement**


### **Iterative prompt engineering**
* No prompt can be perfect at the beginning
* Prompt Engineering is an iterative process where we:
  * Build a prompt
  * Feed it to the model
  * Observe and analyze the output
  * Reiterate to make the prompt better

Sometimes, we must refine our prompt because the model misunderstands the request. For example, asking it to generate an Excel sheet with five students' names and grades might prompt it to create an actual Excel file, which it can't do. Refining the prompt to ask for a table that we can copy into Excel gives us a better output.

  <div style="display: flex;">
    <!-- Left Column -->
    <div style="width: 45%; padding: 10px;">
    <b>Initial prompt</b>
    <div>
    <img src='./images/initial-prompt.png' width=75%>
    </div>
    </div>
    <!-- Right Column -->
    <div style="width: 48%; padding: 10px;">
    <b>Refined prompt</b>
    <div>
    <img src='./images/refined-prompt.png' width=75%>
    </div>
    </div>
</div>

### **Prompt refinement for various prompt types**

Prompt refinement applies to all prompt types. 

* Few-shot prompts: refine examples
* Multi-step prompts: refine guiding steps
* Chain-of-thought and self-consistency prompts: refine problem description